In [1]:
import sys
sys.path.insert(0, '../')
import os
from cycle_prediction.t2e import t2e
from cycle_prediction.weibull_utils import check_dir
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
DEBUG 2021-06-11 17:06:12 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [15]:
path = '../data/'
check_dir(path)

a = os.path.join(path, 'helpdesk.csv')
b = os.path.join(path, 'bpi_12_w.csv')
c = os.path.join(path, 'bpi_12_w_no_repeat.csv')
d = os.path.join(path, 'env_permit.csv')
e = os.path.join(path, 'bpic13_all.csv')
f = os.path.join(path, 'Sepsis_Cases_processed.csv')
g = os.path.join(path, 'road_traffic.csv')
h = os.path.join(path, 'bpic15_2.csv')

a = pd.read_csv(a)
b = pd.read_csv(b)
c = pd.read_csv(c)
d = pd.read_csv(d)
e = pd.read_csv(e)
f = pd.read_csv(f)
g = pd.read_csv(g)
h = pd.read_csv(h)

df_name = {
    'a': 'helpdesk',
    'b': 'bpi12',
    'c': 'bpi12_norep',
    'd': 'env',
    'e': 'bpic13_all',
    'f': 'Sepsis',
    'g': 'Road_traffic',
    'h': 'bpic15'
}

df_dict = {
    'a': a,
    'b': b,
    'c': c,
    'd': d,
    'e': e,
    'f': f,
    'g': g,
    'h': h,
}

range_dict = {
    'a': range(2, 8, 1),
    'b': range(2, 22, 2),
    'c': range(2, 12, 2),
    'd': range(2, 22, 2),
    'e': range(2, 22, 2),
    'f': range(3, 11, 1),
    'g': range(1, 6, 1),
    'h': range(5, 30, 5)
}

end_event_dict = {
    'a': [1,2,3,4,5,6],
    'b': [1,2,3,4,5,6],
    'c': [1,2,3,4,5,6],
    'd': [6],
    'e': ['Completed+Closed', 'Completed+In Call', 'Completed-Closed',
          'Completed+Resolved', 'Completed+Cancelled', 'Completed-Cancelled'],
    'f': ['Release A', 'Release B', 'Release C', 'Release D', 'Release E',
          'Return ER'],
    'g': ['Payment', 'Send for Credit Collection'],
    'h': ['01_HOOFD', '01_BB', '08_AWB45', '05_EIND', '13_CRD', '14_VRIJ',
          '10_UOV', '99_NOCODE', '04_BPT', '06_VD', '12_AP', '01_OLO',
          '02_DRZ', '16_LGSD', '09_AH', '16_LGSV', '03_VD', '03_GBH',
          '07_OPS', '10_OLO', '11_AH']
}

cols = ["prefix", "Layer_Size", "MAE", "unique_pred", "train_size",
        "val_size", "test_size", "Censored %", "fit_time"]

exp_dict = {
    1: {
      'mae_path': '../output/maes/prc/',
      'dynamic_features': ['ActivityID'],
      'static_features': [],
      'transform': 'log'
    },
}

In [38]:
dataset = 'd'
res = 'd'
fit_type = 't2e'
censored = True
size_dyn = 4
extra_censored = 1
prefix = 2

for exp, v in exp_dict.items():
    mae_path = v['mae_path']
    check_dir(mae_path + dataset + '/')
    
    grid_results = pd.DataFrame(columns=cols)
    t2e_obj = t2e(
        dataset=df_dict[dataset],
        prefix=prefix,
        resolution=res,
        dynamic_features=exp_dict[exp]['dynamic_features'],
        static_features=exp_dict[exp]['static_features'],
        fit_type='t2e',
        transform=exp_dict[exp]['transform'],
        end_event_list=df_dict[dataset].drop_duplicates(subset=['CaseID','ActivityID'], keep='last')['ActivityID'].unique().tolist(),
        censored=censored
    )
    t2e_obj.train_val_test_split()
    t2e_obj.preprocess(extra_censored=extra_censored)
    X_train, X_val, X_test, y_train, y_val, y_test = t2e_obj.xy_split()

INFO 2021-06-11 20:47:18 - ========================================================
INFO 2021-06-11 20:47:18 - Prefix = 2
INFO 2021-06-11 20:47:18 - Total cases: 937
INFO 2021-06-11 20:47:18 - Prefix cases: 935
INFO 2021-06-11 20:47:18 - ========================
INFO 2021-06-11 20:47:18 - Extra censored percentage 1
INFO 2021-06-11 20:47:18 - len(df) 360
INFO 2021-06-11 20:47:18 - Extra censored examples 360
INFO 2021-06-11 20:47:19 - TRAINING SET
INFO 2021-06-11 20:47:19 - Censored cases: 359
INFO 2021-06-11 20:47:19 - Observed cases: 1
INFO 2021-06-11 20:47:20 - Y_label has been transformed to logarithmic scale
INFO 2021-06-11 20:47:20 - ========================
INFO 2021-06-11 20:47:21 - VALIDATION SET
INFO 2021-06-11 20:47:21 - Observed cases: 294
INFO 2021-06-11 20:47:22 - Y_label has been transformed to logarithmic scale
INFO 2021-06-11 20:47:22 - ========================
INFO 2021-06-11 20:47:22 - TEST SET
INFO 2021-06-11 20:47:22 - Observed cases: 281
INFO 2021-06-11 20:47:23 -

In [39]:
t2e_obj.build_model(X_train, y_train,
                        size_dyn=8, size_sta=4)

INFO 2021-06-11 20:47:27 - Initializing time to event model ...


In [40]:
t2e_obj.fit(X_train, y_train, X_val, y_val,
                bs=128, exp_dir=dataset+'_'+str(exp)+'_'+str(prefix),
                vb=False)

INFO 2021-06-11 20:47:29 - No Static features added
INFO 2021-06-11 20:47:29 - Variable Input Shape: (360, 2, 150)
INFO 2021-06-11 20:47:29 - Output   Input Shape: (360, 2)
INFO 2021-06-11 20:47:29 - Fitting model ... Batch size: 128


In [ ]:
try:
    t2e_obj.build_model(X_train, y_train,
                        size_dyn=8, size_sta=4)
    t2e_obj.fit(X_train, y_train, X_val, y_val,
                bs=128, exp_dir=dataset+'_'+str(exp)+'_'+str(prefix),
                vb=False)
except Exception:
    try:
        t2e_obj.build_model(X_train, y_train,
                            size_dyn=8, size_sta=4)
        t2e_obj.fit(X_train, y_train, X_val, y_val,
                    bs=64,
                    exp_dir=dataset+'_'+str(exp)+'_'+str(prefix),
                    vb=False)
    except Exception:
        t2e_obj.build_model(X_train, y_train,
                            size_dyn=8, size_sta=4)
        t2e_obj.fit(X_train, y_train, X_val, y_val,
                    bs=32,
                    exp_dir=dataset+'_'+str(exp)+'_'+str(prefix),
                    vb=False)





In [41]:
test_result_df, mae = t2e_obj.evaluate(X_test, y_test)
nunique = test_result_df["T_pred"].nunique()
cen_percentage = t2e_obj.get_cen_prc()
grid_results = grid_results.append(
    pd.DataFrame([[prefix, size_dyn, mae, nunique, len(X_train[0]),
                   len(X_val[0]), len(X_test[0]), cen_percentage,
                   t2e_obj.fit_time]], columns=cols),
    ignore_index=True)
    

grid_results



INFO 2021-06-11 20:55:16 - Evaluating test set ...
INFO 2021-06-11 20:55:16 - Y_label is restored


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
pickle.dump(grid_results, open(mae_path + 
                dataset + '/' + str(extra_censored)+
                '_GRU.pkl', 'wb'))